In [1]:
import pandas as pd
import numpy as np
import glob
import tqdm
import string
import multiprocessing as mp
import os

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

sw = stopwords.words('spanish')

#from thefuzz import fuzz
#from thefuzz import process

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed

from datetime import datetime

/Users/cevalenciam/anaconda3/envs/smartbuy/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
mp.cpu_count()

8

In [3]:
os.getcwd()

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/homologacion-smartbuy'

In [4]:
ruta = os.getcwd()

### Productos

In [15]:
def marca_tottus(x):
    if x.find('marca: ')==-1:
        return('')
    elif x.find(';', x.find('marca: ')+1)==-1:
        return(x[x.find('marca: ')+7:])
    else:
        return(x[x.find('marca: ')+7: x.find(';', x.index('marca: ')+1)])

In [16]:
def formato_tottus(x):
    if x.find('formato: ')==-1:
        return('')
    elif x.find(';', x.find('formato: ')+1)==-1:
        return(x[x.find('formato: ')+9:])
    else:
        return(x[x.find('formato: ')+9: x.find(';', x.find('formato: ')+1)])

In [35]:
def format(x):
    if x[-2:]=='gr':
        return(x[:-1])
    elif x[-2:]=='lt':
        return(x[:-1])
    else:
        return(x)

In [26]:
df_tottus = pd.read_excel("../web-scraper-tottus/Output/REPORTE_WS_TOTTUS.xlsx", converters={'internet_price':str})
df_tottus['name'] = df_tottus['name'].astype(str).replace('nan', 'Sin información')
df_tottus['description'] = df_tottus['description'].astype(str).map(lambda x:x[x.find('>', x.find('h3'))+1:x.find('</h3>')] if x[0]=='<' else x).replace('nan', 'Sin información')
df_tottus['marca'] = df_tottus['attributes'].astype(str).str.lower().map(lambda x:marca_tottus(x)).replace('nan', 'Sin información')
df_tottus['formato'] = df_tottus['attributes'].astype(str).str.lower().map(lambda x:formato_tottus(x)).replace('nan', 'Sin información')
df_tottus['formato'] = df_tottus['formato'].map(lambda x:format(x))
df_tottus['image1'] = df_tottus['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_tottus['image2'] = df_tottus['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'Sin información')
df_tottus['image3'] = df_tottus['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_tottus['internet_price'] = df_tottus['internet_price'].astype(str).str.replace('.', '').str.replace('nan', '0').astype(float)
df_tottus['empresa'] = 'Tottus'
df_tottus.head(3)

/var/folders/pt/7bqwwkcs24v759tlk02fj1m80000gn/T/ipykernel_49837/1761821536.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_tottus['internet_price'] = df_tottus['internet_price'].astype(str).str.replace('.', '').str.replace('nan', '0').astype(float)


,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,...,cmr_price,url_image,pixel_image,ID_DIARUNNING,marca,formato,image1,image2,image3,empresa
0,CL,https://tottus.falabella.com/tottus-cl/product...,110606885.0,ACEITE VEGETAL TOTTUS 900 CC,ACEITE VEGETAL TOTTUS 900 CC,110606886.0,20416961.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/204169...,"(400, 400)",2023-06-11 15:36:45.583,,900 ml,https://s7d2.scene7.com/is/image/Tottus/204169...,,,Tottus
1,CL,https://tottus.falabella.com/tottus-cl/product...,110615978.0,Aceite Vegetal,Aceite Vegetal,110615979.0,1020028.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/010200...,"(400, 400)",2023-06-11 15:36:45.583,belmont,1 l,https://s7d2.scene7.com/is/image/Tottus/010200...,,,Tottus
2,CL,https://tottus.falabella.com/tottus-cl/product...,110607018.0,Aceite Oliva Extra Virgen - 500 ML,Aceite de Oliva Extra Virgen Tottus,110607020.0,20109158.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/201091...,"(400, 400)",2023-06-11 15:36:45.583,tottus,500 ml,https://s7d2.scene7.com/is/image/Tottus/201091...,,,Tottus


In [28]:
df_tottus.shape

(6330, 25)

In [29]:
df_tottus.groupby('first_category').size()

first_category
Bebés y Niños            641
Despensa                1764
Electro y Tecnología     488
Hogar y Aire Libre      1132
Líquidos                 611
Vestuario y Deporte     1247
Vive Tu Espacio          446
dtype: int64

In [85]:
def formato_lider(x):
    if x[x.rfind(',')+1:x.rfind(',')+2]!=' ':
        return(x[x.rfind(',', 0, x.rfind(','))+2:])
    else:
        return(x[x.rfind(',')+2:])

In [86]:
filenames = glob.glob("../web-scraper-lider/Output/REPORTE_WS_*")
df_lider = pd.concat(pd.read_excel(file, converters={'internet_price':str}) for file in filenames).reset_index(drop=True)
df_lider['name'] = df_lider['name'].astype(str).replace('nan', 'Sin información')
df_lider['description'] = df_lider['description'].astype(str).replace('nan', 'Sin información')
df_lider['marca'] = df_lider['attributes'].astype(str).str.lower().str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u').replace('nan', 'Sin información')
df_lider['formato'] = df_lider['name'].astype(str).str.lower().map(lambda x:formato_lider(x)).replace('nan', 'Sin información')
df_lider['formato'] = df_lider['formato'].map(lambda x:format(x))
df_lider['image1'] = df_lider['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_lider['image2'] = df_lider['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'Sin información')
df_lider['image3'] = df_lider['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_lider['internet_price'] = np.where(df_lider['internet_price'].astype(str).map(lambda x:x[0].isdigit()), df_lider['normal_price'], df_lider['internet_price'])
df_lider['internet_price'] = df_lider['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)
df_lider['empresa'] = 'Lider'
df_lider.head(3)

/var/folders/pt/7bqwwkcs24v759tlk02fj1m80000gn/T/ipykernel_49837/2681619207.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_lider['internet_price'] = df_lider['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)


,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,...,cmr_price,url_image,pixel_image,ID_DIARUNNING,marca,formato,image1,image2,image3,empresa
0,CL,https://www.lider.cl/supermercado/product/sku/...,item 867428,"Leche en Polvo Etapa 1+ Bolsa, 2,2 kg",Fórmula Láctea de Crecimiento en Polvo Instant...,item 867428,item 867428,Mundo Bebé y Juguetería,Alimentación y Lactancia,Leches y Fórmulas,...,NaN,https://images.lider.cl/wmtcl?source=url[file:...,"(800, 800)",2023-06-14 13:14:22.611,nido,"2,2 kg",https://images.lider.cl/wmtcl?source=url[file:...,,,Lider
1,CL,https://www.lider.cl/supermercado/product/sku/...,item 558411,"Pediasure Chocolate Tarro 900 grs, 1 Un",Complemento Nutricional en Polvo Pediasure Tar...,item 558411,item 558411,Mundo Bebé y Juguetería,Alimentación y Lactancia,Leches y Fórmulas,...,NaN,https://images.lider.cl/wmtcl?source=url[file:...,"(800, 800)",2023-06-14 13:14:22.611,pediasure,1 un,https://images.lider.cl/wmtcl?source=url[file:...,,,Lider
2,CL,https://www.lider.cl/supermercado/product/sku/...,item 558412,"Pediasure Vanilla Tarro 900 grs, 1 Un",Complemento Nutricional para Niños Vainilla Pe...,item 558412,item 558412,Mundo Bebé y Juguetería,Alimentación y Lactancia,Leches y Fórmulas,...,NaN,https://images.lider.cl/wmtcl?source=url[file:...,"(800, 800)",2023-06-14 13:14:22.611,pediasure,1 un,https://images.lider.cl/wmtcl?source=url[file:...,,,Lider


In [37]:
df_lider.shape

(57380, 25)

In [38]:
df_lider.groupby('first_category').size()

first_category
Automóvil                   378
Bebidas y Licores          3798
Campañas                   9674
Carnes y Pescados          2246
Colaciones                 1033
Congelados                 1252
Deporte y Aire Libre        402
Desayunos y Dulces         3900
Despensa                   3831
Ferretería                  707
Frescos y Lácteos          3517
Frutas y Verduras           732
Hogar                      2404
Librería y Cumpleaños      2166
Limpieza y Aseo            2884
Marcas Propias             3292
Mascotas                    912
Mundo Bebé y Juguetería    2873
Panadería y Pastelería     1342
Parrillas y Jardín          263
Perfumería y Salud         5992
Platos Preparados           478
Tecno y Electro             785
Vestuario                  2470
dtype: int64

In [90]:
filenames = glob.glob("../web-scraper-jumbo/Output/REPORTE_WS_*")
df_jumbo = pd.concat(pd.read_excel(file, converters={'internet_price':str}) for file in filenames).reset_index(drop=True)
df_jumbo['name'] = df_jumbo['name'].astype(str).replace('nan', 'Sin información')
df_jumbo['description'] = df_jumbo['description'].astype(str).replace('nan', 'Sin información')
df_jumbo['marca'] = df_jumbo['attributes'].astype(str).str.lower().str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u').replace('nan', 'Sin información')
df_jumbo['formato'] = df_jumbo['name'].astype(str).str.lower().map(lambda x:x[x[:x.rfind(" ")].rfind(" ")+1:]).replace('nan', 'Sin información')
df_jumbo['formato'] = df_jumbo['formato'].map(lambda x:format(x))
df_jumbo['image1'] = df_jumbo['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_jumbo['image2'] = df_jumbo['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'Sin información')
df_jumbo['image3'] = df_jumbo['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'Sin información')
df_jumbo['internet_price'] = df_jumbo['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)
df_jumbo['empresa'] = 'Jumbo'
df_jumbo.head(3)

/var/folders/pt/7bqwwkcs24v759tlk02fj1m80000gn/T/ipykernel_49837/2193528619.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_jumbo['internet_price'] = df_jumbo['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)


,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,...,cmr_price,url_image,pixel_image,ID_DIARUNNING,marca,formato,image1,image2,image3,empresa
0,CL,https://www.jumbo.cl/alim-perro-one-cachorro-1...,Código: 1864398,Alimento perro cachorro pollo y carne 12 kg,Alimento seco para cachorros y hembras gestant...,Código: 1864398,Código: 1864398,Mascotas,Perros,Alimentos Cachorros,...,NaN,https://jumbo.vtexassets.com/arquivos/ids/4805...,"(750, 750)",2023-06-08 07:21:32.841,purina one,12 kg,https://jumbo.vtexassets.com/arquivos/ids/4805...,,,Jumbo
1,CL,https://www.jumbo.cl/alimento-para-perro-maste...,Código: 1726893,Alimento perro cachorro raza mediana y grande ...,Alimento completo y balanceado para tu cachorr...,Código: 1726893,Código: 1726893,Mascotas,Perros,Alimentos Cachorros,...,NaN,https://jumbo.vtexassets.com/arquivos/ids/5915...,"(750, 750)",2023-06-08 07:21:32.841,master dog,15 kg,https://jumbo.vtexassets.com/arquivos/ids/5915...,,,Jumbo
2,CL,https://www.jumbo.cl/alim-perro-champion-cacho...,Código: 1627598,"Alimento perro cachorro carne, pollo y leche 1...",La felicidad de tu perro se alimenta todos los...,Código: 1627598,Código: 1627598,Mascotas,Perros,Alimentos Cachorros,...,NaN,https://jumbo.vtexassets.com/arquivos/ids/3732...,"(750, 750)",2023-06-08 07:21:32.841,champion dog,15 kg,https://jumbo.vtexassets.com/arquivos/ids/3732...,,,Jumbo


In [91]:
df_jumbo.shape

(30573, 25)

In [92]:
df_jumbo.groupby('first_category').size()

first_category
Botillería            4135
Carnicería             413
Despensa              3428
Frutas y Verduras      631
Limpieza              2591
Lácteos               1073
Mascotas               652
Supermercado         17590
dtype: int64

In [96]:
df_productos = pd.concat([  df_tottus[df_tottus['first_category'].isin(['Despensa', 'Líquidos'])], 
                            df_lider[df_lider['first_category'].isin(['Bebidas y Licores', 'Carnes y Pescados', 'Despensa', 'Frescos y Lácteos', 'Frutas y Verduras'])], 
                            df_jumbo[df_jumbo['first_category'].isin(['Botillería', 'Carnicería', 'Despensa', 'Frutas y Verduras', 'Lácteos'])]]).reset_index(drop=True)
df_productos.head(3)

,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,...,cmr_price,url_image,pixel_image,ID_DIARUNNING,marca,formato,image1,image2,image3,empresa
0,CL,https://tottus.falabella.com/tottus-cl/product...,110606885.0,ACEITE VEGETAL TOTTUS 900 CC,ACEITE VEGETAL TOTTUS 900 CC,110606886.0,20416961.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/204169...,"(400, 400)",2023-06-11 15:36:45.583,,900 ml,https://s7d2.scene7.com/is/image/Tottus/204169...,,,Tottus
1,CL,https://tottus.falabella.com/tottus-cl/product...,110615978.0,Aceite Vegetal,Aceite Vegetal,110615979.0,1020028.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/010200...,"(400, 400)",2023-06-11 15:36:45.583,belmont,1 l,https://s7d2.scene7.com/is/image/Tottus/010200...,,,Tottus
2,CL,https://tottus.falabella.com/tottus-cl/product...,110607018.0,Aceite Oliva Extra Virgen - 500 ML,Aceite de Oliva Extra Virgen Tottus,110607020.0,20109158.0,Despensa,Abarrotes,Aceites,...,NaN,https://s7d2.scene7.com/is/image/Tottus/201091...,"(400, 400)",2023-06-11 15:36:45.583,tottus,500 ml,https://s7d2.scene7.com/is/image/Tottus/201091...,,,Tottus


In [97]:
df_productos.groupby(['empresa', 'first_category']).size()

empresa  first_category   
Jumbo    Botillería           4135
         Carnicería            413
         Despensa             3428
         Frutas y Verduras     631
         Lácteos              1073
Lider    Bebidas y Licores    3798
         Carnes y Pescados    2246
         Despensa             3831
         Frescos y Lácteos    3517
         Frutas y Verduras     732
Tottus   Despensa             1764
         Líquidos              611
dtype: int64

In [98]:
vars_productos = ['name', 'description', 'first_category', 'second_category', 'third_category', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3', 'empresa']

In [99]:
df = df_productos[vars_productos].reset_index(drop=True)
df.head(3)

,name,description,first_category,second_category,third_category,marca,formato,internet_price,image1,image2,image3,empresa
0,ACEITE VEGETAL TOTTUS 900 CC,ACEITE VEGETAL TOTTUS 900 CC,Despensa,Abarrotes,Aceites,,900 ml,2000.0,https://s7d2.scene7.com/is/image/Tottus/204169...,,,Tottus
1,Aceite Vegetal,Aceite Vegetal,Despensa,Abarrotes,Aceites,belmont,1 l,3940.0,https://s7d2.scene7.com/is/image/Tottus/010200...,,,Tottus
2,Aceite Oliva Extra Virgen - 500 ML,Aceite de Oliva Extra Virgen Tottus,Despensa,Abarrotes,Aceites,tottus,500 ml,5490.0,https://s7d2.scene7.com/is/image/Tottus/201091...,,,Tottus


In [100]:
print("Total: ",df.shape)
df = df.drop_duplicates()
print("Total sin duplicados: ",df.shape)
df.head(3)

Total:  (26179, 12)
Total sin duplicados:  (26179, 12)


,name,description,first_category,second_category,third_category,marca,formato,internet_price,image1,image2,image3,empresa
0,ACEITE VEGETAL TOTTUS 900 CC,ACEITE VEGETAL TOTTUS 900 CC,Despensa,Abarrotes,Aceites,,900 ml,2000.0,https://s7d2.scene7.com/is/image/Tottus/204169...,,,Tottus
1,Aceite Vegetal,Aceite Vegetal,Despensa,Abarrotes,Aceites,belmont,1 l,3940.0,https://s7d2.scene7.com/is/image/Tottus/010200...,,,Tottus
2,Aceite Oliva Extra Virgen - 500 ML,Aceite de Oliva Extra Virgen Tottus,Despensa,Abarrotes,Aceites,tottus,500 ml,5490.0,https://s7d2.scene7.com/is/image/Tottus/201091...,,,Tottus


In [101]:
df[df['empresa']=='Tottus'].shape, df[df['empresa']=='Lider'].shape, df[df['empresa']=='Jumbo'].shape

((2375, 12), (14124, 12), (9680, 12))

In [102]:
df['product'] = df['name'].astype(str) + ' ' +\
                df['marca'].astype(str) + ' ' +\
                df['formato'].astype(str)

In [103]:
lines = df['product'].tolist()

In [104]:
# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]

def remove_stopwords(lines, sw = sw):
    '''
    The purpose of this function is to remove stopwords from a given array of 
    lines.
    
    params:
        lines (Array / List) : The list of lines you want to remove the stopwords from
        sw (Set) : The set of stopwords you want to remove
        
    example:
        lines = remove_stopwords(lines = lines, sw = sw)
    '''
    
    res = []
    for line in lines:
        original = line
        line = [w for w in line if w not in sw]
        if len(line) < 1:
            line = original
        res.append(line)
    return res
    
filtered_lines = remove_stopwords(lines = lines, sw = sw)

filtered_lines = [TreebankWordDetokenizer().detokenize(line) for line in filtered_lines]

In [105]:
filtered_lines[1]

'aceite vegetal belmont 1 l'

In [106]:
df['product'] = filtered_lines

In [107]:
collection_tottus = df[df['empresa']=='Tottus']['product'].values
collection_lider = df[df['empresa']=='Lider']['product'].values
collection_jumbo = df[df['empresa']=='Jumbo']['product'].values

In [108]:
def matching(product, empresa, collection_tottus, collection_lider, collection_jumbo):
    tottus = process.extractOne(product, collection_tottus, scorer=fuzz.ratio)
    lider = process.extractOne(product, collection_lider, scorer=fuzz.ratio)
    jumbo = process.extractOne(product, collection_jumbo, scorer=fuzz.ratio)
    return([product, empresa, tottus[0], tottus[1], lider[0], lider[1], jumbo[0], jumbo[1]])

In [109]:
products = df[['product', 'empresa']].drop_duplicates().values

In [110]:
results = Parallel(n_jobs=-1, prefer="threads")(delayed(matching)(product, empresa, collection_tottus, collection_lider, collection_jumbo) for product, empresa in tqdm.tqdm(products))

df_res = pd.DataFrame(results, columns=['product', 'empresa', 'name_tottus', 'score_tottus', 'name_lider', 'score_lider', 'name_jumbo', 'score_jumbo'])

  0%|          | 0/13409 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df_res_0 = pd.merge(df_res, df[['product', 'empresa', 'name', 'marca', 'formato']].drop_duplicates(), how='left', on=['product', 'empresa'])
df_res_0 = df_res_0[['name', 'marca', 'formato', 'product', 'empresa', 'name_tottus', 'score_tottus', 'name_lider', 'score_lider', 'name_jumbo', 'score_jumbo']]

In [ ]:
df_res_1 = pd.merge(df_res_0, df[['product', 'marca', 'formato', 'internet_price', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_tottus',
                                                                                                                                                            'marca': 'marca_tottus',
                                                                                                                                                            'formato': 'formato_tottus',
                                                                                                                                                            'internet_price': 'precio_tottus',
                                                                                                                                                            'first_category': 'first_category_tottus',
                                                                                                                                                            'second_category': 'second_category_tottus',
                                                                                                                                                            'third_category': 'third_category_tottus'}), how='left', on='name_tottus')

In [ ]:
df_res_2 = pd.merge(df_res_1, df[['product', 'marca', 'formato', 'internet_price', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_lider',
                                                                                                                                                            'marca': 'marca_lider',
                                                                                                                                                            'formato': 'formato_lider',
                                                                                                                                                            'internet_price': 'precio_lider',
                                                                                                                                                            'first_category': 'first_category_lider',
                                                                                                                                                            'second_category': 'second_category_lider',
                                                                                                                                                            'third_category': 'third_category_lider'}), how='left', on='name_lider')

In [ ]:
df_res_3 = pd.merge(df_res_2, df[['product', 'marca', 'formato', 'internet_price', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_jumbo',
                                                                                                                                                            'marca': 'marca_jumbo',
                                                                                                                                                            'formato': 'formato_jumbo',
                                                                                                                                                            'internet_price': 'precio_jumbo',
                                                                                                                                                            'first_category': 'first_category_jumbo',
                                                                                                                                                            'second_category': 'second_category_jumbo',
                                                                                                                                                            'third_category': 'third_category_jumbo'}), how='left', on='name_jumbo')

In [ ]:
df_final = df_res_3[['name', 'marca', 'formato', 'empresa', 'name_tottus', 'score_tottus', 'marca_tottus', 'formato_tottus', 'precio_tottus', 'name_lider', 'score_lider', 'marca_lider', 'formato_lider', 'precio_lider', 'name_jumbo', 'score_jumbo', 'marca_jumbo', 'formato_jumbo', 'precio_jumbo']].drop_duplicates()
df_final.head(3)

,name,marca,formato,empresa,name_tottus,score_tottus,marca_tottus,formato_tottus,precio_tottus,name_lider,score_lider,marca_lider,formato_lider,precio_lider,name_jumbo,score_jumbo,marca_jumbo,formato_jumbo,precio_jumbo
0,ACEITE MAIZ 1 LT. CH,chef,1 LT,Tottus,aceite maiz 1 lt ch chef 1 lt,100,chef,1 LT,5200.0,aceite maiz botella 1 l chef 1 l,79,chef,1 L,5090.0,aceite maíz 1 l chef,78,chef,,5199.0
2,Aceite de Oliva Extra Virgen - 500 ML,tottus,500 ML,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ML,5490.0,aceite oliva extra virgen botella 500 ml olave...,83,olave,500 ml,6550.0,aceite oliva extra virgen 500 ml trattoria,82,trattoria,,5299.0
4,SALSA TOSCANA AL PESTO 200 GRS..,,200 GR,Tottus,salsa toscana pesto 200 grs 200 gr,100,,200 GR,940.0,salsa tomates preparada pesto 200 g lider 200 g,74,lider,200 g,680.0,salsa tomate pisto 300 g helios,62,helios,,1899.0


In [ ]:
df_final.shape

(10, 19)

In [ ]:
df_final[['name', 'empresa']].drop_duplicates().shape

(10, 2)

In [ ]:
actual_date = str(datetime.now())[0:10]

df_final.to_excel(ruta + f'df_matching.xlsx'.format(actual_date), index=0)

In [161]:
x =  '900 ML'

In [162]:
x[-2:]

'ML'

In [160]:
df_final['formato_tottus'].unique()

array(['900 ML', '1 LT', '500 ML', '5 LT', '400 GR', '200 GR', '250 GR',
       '1,4 KG', '296 GR', '6 UN X 200 GR', '500 GR', '1 KG', '170 GR',
       '567 GR', '160 GR', '380 GR', '130 GR', '735 GR', '900 GR',
       '850 GR', '760 GR', '950 GR', nan, '750 GR', '0,5 GR', '15 GR',
       '350 GR', '700 GR', '50 GR', 'UN', '22 GR', '100 GR', '200 PRS',
       '110 GR', '150 GR', '120 GR', '300 GR', '150 ML', '270 GR',
       '120 gr', '182 GR', '80 GR', '750 CC', '130 g', '216 GR', '180 GR',
       '85 GR', '40 GR', '585 GR', '95 GR', '140 GR', '378 GR', '255 GR',
       '126 GR', '107 GR', '134 GR', '185 GR', '3 KG', '285 GR', '220 GR',
       '300 gr', '430 gr', '128 GR', '192 GR', '112 GR', '133 GR',
       '172 GR', '16 UN', 'un', '152 GR', '170 gr', '184 GR', '162 GR',
       '102,4 GR', '144 GR', '125 GR', '100 UN', '350 ML', '250 ML',
       '6 UN X 21 GR', '342 GR', '450 GR', '346 GR', '480 GR', '370 GR',
       '630 GR', '730 GR', '720 GR', '8 UN', '20 GR', '12 x 1 LT',
      

In [163]:
df_final['formato_lider'].unique()

array(['900 ml', '1 L', '500 ml', ..., '75 cc', '60 cc', '300 grs'],
      dtype=object)

In [164]:
df_final['formato_lider'].unique()

array(['900 ml', '1 L', '500 ml', ..., '75 cc', '60 cc', '300 grs'],
      dtype=object)

In [159]:
np.where(df_final[''])

,name,marca,formato,empresa,name_tottus,score_tottus,marca_tottus,formato_tottus,precio_tottus,name_lider,...,marca_lider,formato_lider,precio_lider,name_jumbo,score_jumbo,marca_jumbo,formato_jumbo,precio_jumbo,Unnamed: 19,Unnamed: 20
0,ACEITE VEGETAL TOTTUS 900 CC,NaN,900 ML,Tottus,aceite vegetal tottus 900 cc 900 ml,100,NaN,900 ML,2000,aceite vegetal 900 ml romana 900 ml,...,la romana,900 ml,2890,aceite vegetal 900 ml natura 900 ml,74,natura,900 ml,2849,NaN,0.014391
1,Aceite Vegetal,belmont,1 LT,Tottus,aceite vegetal belmont 1 lt,100,belmont,1 LT,3940,aceite vegetal botella 1 l belmont 1 l,...,belmont,1 L,2590,aceite vegetal 1 l belmont 1 l,91,belmont,1 L,3299,NaN,0.214914
2,Aceite Oliva Extra Virgen - 500 ML,tottus,500 ML,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ML,5490,aceite oliva extra virgen botella 500 ml olave...,...,olave,500 ml,6550,aceite oliva extra virgen 500 ml trattoria 500 ml,91,trattoria,500 ml,5299,NaN,0.236082
3,Aceite de Oliva Extra Virgen - 500 ML,tottus,500 ML,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ML,5490,aceite oliva extra virgen botella 500 ml olave...,...,olave,500 ml,6550,aceite oliva extra virgen 500 ml trattoria 500 ml,91,trattoria,500 ml,5299,NaN,0.236082
4,ACEITE VEGETAL TOTTUS 5 LT,tottus,5 LT,Tottus,aceite vegetal tottus 5 lt tottus 5 lt,100,tottus,5 LT,10000,aceite vegetal bidon 5 l acuenta 5 l,...,acuenta,5 L,9890,aceite vegetal 1 l belmont 1 l,62,belmont,1 L,3299,NaN,1.997878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44600,Trutro entero de pollo 500 g,granja m,500 g,Jumbo,palmitos enteros 400 gr tottus 400 gr,52,tottus,400 GR,2990,trutro entero pollo deshuesado 800 g seara 800 g,...,seara,800 g,3190,trutro entero pollo 500 g granja m 500 g,100,granja m,500 g,6890,NaN,0.537010
44601,Pollo Entero con Menudencias Don Pollo envasado,don pollo,Pollo envasado,Jumbo,polera color entero cuello redondo talla s all...,50,allbasics,UN,3990,pollo entero menudencias envasado 25 kg super ...,...,super pollo,"2,5 kg",9475,pollo entero menudencias don pollo envasado do...,100,don pollo,Pollo envasado,6980,NaN,0.357450
44602,Corazon de Pollo 500 gr,ariztia,500 gr,Jumbo,café molido 500 gr tottus 500 gr,66,tottus,500 GR,6290,contre pollo 480 g ariztia 480 g,...,ariztia,480 g,1590,corazon pollo 500 gr ariztia 500 gr,100,ariztia,500 gr,2190,NaN,0.273973
44603,Contre de Pollo 480 gr,ariztia,480 gr,Jumbo,leche polvo calo crecer 1 800 grs calo 800 gr,56,calo,800 GR,6230,contre pollo 480 g ariztia 480 g,...,ariztia,480 g,1590,contre pollo 480 gr ariztia 480 gr,100,ariztia,480 gr,1990,NaN,0.201005


In [25]:
umbral = 70

In [142]:
df_final[df_final['empresa']=='Tottus'].shape, df_final[df_final['empresa']=='Lider'].shape, df_final[df_final['empresa']=='Jumbo'].shape

((3545, 21), (21120, 21), (19940, 21))

In [28]:
# Tottus vs Lider
df_final[(df_final['empresa']=='Tottus') & (df_final['score_lider']>=umbral)].shape[0]

1463

In [27]:
# Tottus vs Lider
df_final[(df_final['empresa']=='Tottus') & (df_final['score_lider']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Tottus')].shape[0]

0.41269393511988717

In [29]:
# Tottus vs Jumbo
df_final[(df_final['empresa']=='Tottus') & (df_final['score_jumbo']>=umbral)].shape[0]

1327

In [ ]:
# Tottus vs Jumbo
df_final[(df_final['empresa']=='Tottus') & (df_final['score_jumbo']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Tottus')].shape[0]

0.37433004231311706

In [30]:
# Lider vs Tottus
df_final[(df_final['empresa']=='Lider') & (df_final['score_tottus']>=umbral)].shape[0]

4715

In [ ]:
# Lider vs Tottus
df_final[(df_final['empresa']=='Lider') & (df_final['score_tottus']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Lider')].shape[0]

0.22324810606060605

In [31]:
# Lider vs Jumbo
df_final[(df_final['empresa']=='Lider') & (df_final['score_jumbo']>=umbral)].shape[0]

15929

In [ ]:
# Lider vs Jumbo
df_final[(df_final['empresa']=='Lider') & (df_final['score_jumbo']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Lider')].shape[0]

0.7542140151515152

In [32]:
# Jumbo vs Tottus
df_final[(df_final['empresa']=='Jumbo') & (df_final['score_tottus']>=umbral)].shape[0]

4998

In [ ]:
# Jumbo vs Tottus
df_final[(df_final['empresa']=='Jumbo') & (df_final['score_tottus']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Jumbo')].shape[0]

0.2506519558676028

In [33]:
# Jumbo vs Lider
df_final[(df_final['empresa']=='Jumbo') & (df_final['score_lider']>=umbral)].shape[0]

16282

In [ ]:
# Jumbo vs Lider
df_final[(df_final['empresa']=='Jumbo') & (df_final['score_lider']>=umbral)].shape[0]/df_final[(df_final['empresa']=='Jumbo')].shape[0]

0.8165496489468406

### Categorias

In [69]:
df_cat_tottus = pd.read_excel("../web-scraper-tottus/Output/df_categorias_tottus.xlsx")
df_cat_tottus['empresa'] = 'Tottus'
df_cat_tottus.head(3)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus


In [70]:
df_cat_lider = pd.read_excel("../web-scraper-lider/Output/df_categorias_lider.xlsx")
df_cat_lider['empresa'] = 'Lider'
df_cat_lider.head(3)

,first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Marcas Propias,Despensa,https://www.lider.cl/supermercado/category/Mar...,Abarrotes,https://www.lider.cl/supermercado/category/Mar...,Lider
1,Marcas Propias,Despensa,https://www.lider.cl/supermercado/category/Mar...,Pastas y Salsas,https://www.lider.cl/supermercado/category/Mar...,Lider
2,Marcas Propias,Despensa,https://www.lider.cl/supermercado/category/Mar...,Aceites,https://www.lider.cl/supermercado/category/Mar...,Lider


In [71]:
df_cat_jumbo = pd.read_excel("../web-scraper-jumbo/Output/df_categorias_jumbo.xlsx")
df_cat_jumbo['empresa'] = 'Jumbo'
df_cat_jumbo.head(3)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Ofertas,https://www.jumbo.cl/,NaN,NaN,Jumbo Prime,https://www.jumbo.cl/oferta/prime?nombre_promo...,Jumbo
1,Ofertas,https://www.jumbo.cl/,NaN,NaN,Jumbo Ofertas,https://www.jumbo.cl/oferta/oferta-fdm?nombre_...,Jumbo
2,Ofertas,https://www.jumbo.cl/,NaN,NaN,Tarjeta Cencosud,https://www.jumbo.cl/oferta/tarjeta-cencosud?n...,Jumbo


In [72]:
df_categorias = pd.concat([df_cat_tottus, df_cat_lider, df_cat_jumbo]).reset_index(drop=True)
df_categorias.head(3)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus


In [131]:
df_categorias

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus
3,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Conservas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
4,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aderezos y Condimentos,https://tottus.falabella.com/tottus-cl/categor...,Tottus
...,...,...,...,...,...,...,...
1477,Mascotas,https://www.jumbo.cl/mascotas,Gatos,https://www.jumbo.cl/mascotas/gatos,Snacks y Salsas Gatos,https://www.jumbo.cl/mascotas/gatos/snacks-y-s...,Jumbo
1478,Mascotas,https://www.jumbo.cl/mascotas,Gatos,https://www.jumbo.cl/mascotas/gatos,Ver todos,https://www.jumbo.cl/mascotas/gatos,Jumbo
1479,Mascotas,https://www.jumbo.cl/mascotas,Otras Mascotas,https://www.jumbo.cl/mascotas/otras-mascotas,Alimentos Otras Mascotas,https://www.jumbo.cl/mascotas/otras-mascotas/a...,Jumbo
1480,Mascotas,https://www.jumbo.cl/mascotas,Otras Mascotas,https://www.jumbo.cl/mascotas/otras-mascotas,Accesorios e Higiene Otras Mascotas,https://www.jumbo.cl/mascotas/otras-mascotas/a...,Jumbo


In [115]:
df_cat = df_categorias.reset_index(drop=True)
df_cat.head(3)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus


In [116]:
print("Total: ",df_cat.shape)
df_cat = df_cat.drop_duplicates()
print("Total sin duplicados: ",df_cat.shape)
df_cat.head(3)

Total:  (1482, 7)
Total sin duplicados:  (1482, 7)


,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus


In [117]:
df_cat['first_level_category_2'] = df_cat['first_level_category'].str.replace(' ', '-')
df_cat['second_level_category_2'] = df_cat['second_level_category'].str.replace(' ', '-')
df_cat['third_level_category_2'] = df_cat['third_level_category'].str.replace(' ', '-')
df_cat.head(3)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,empresa,first_level_category_2,second_level_category_2,third_level_category_2
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,Tottus,Despensa,Abarrotes,Aceites
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,Tottus,Despensa,Abarrotes,Pastas-y-Salsas
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,Tottus,Despensa,Abarrotes,"Arroz,-Legumbres-y-Semillas"


In [118]:
df_cat[df_cat['empresa']=='Tottus'].shape, df_cat[df_cat['empresa']=='Lider'].shape, df_cat[df_cat['empresa']=='Jumbo'].shape

((289, 10), (904, 10), (289, 10))

In [119]:
df_cat['category'] = df_cat['first_level_category_2'].astype(str) + ' ' +\
                    df_cat['second_level_category_2'].astype(str) + ' ' +\
                    df_cat['third_level_category_2'].astype(str)

In [120]:
lines = df_cat['category'].tolist()

In [121]:
# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]

def remove_stopwords(lines, sw = sw):
    '''
    The purpose of this function is to remove stopwords from a given array of 
    lines.
    
    params:
        lines (Array / List) : The list of lines you want to remove the stopwords from
        sw (Set) : The set of stopwords you want to remove
        
    example:
        lines = remove_stopwords(lines = lines, sw = sw)
    '''
    
    res = []
    for line in lines:
        original = line
        line = [w for w in line if w not in sw]
        if len(line) < 1:
            line = original
        res.append(line)
    return res
    
filtered_lines = remove_stopwords(lines = lines, sw = sw)

filtered_lines = [TreebankWordDetokenizer().detokenize(line) for line in filtered_lines]

In [122]:
filtered_lines[1]

'despensa abarrotes pastasysalsas'

In [123]:
lines

[['despensa', 'abarrotes', 'aceites'],
 ['despensa', 'abarrotes', 'pastasysalsas'],
 ['despensa', 'abarrotes', 'arrozlegumbresysemillas'],
 ['despensa', 'abarrotes', 'conservas'],
 ['despensa', 'abarrotes', 'aderezosycondimentos'],
 ['despensa', 'abarrotes', 'vertodo'],
 ['despensa', 'dulcesysnack', 'chocolates'],
 ['despensa', 'dulcesysnack', 'galletas'],
 ['despensa', 'dulcesysnack', 'papasfritas'],
 ['despensa', 'dulcesysnack', 'cocktailysnack'],
 ['despensa', 'dulcesysnack', 'vertodo'],
 ['despensa', 'desayunos', 'café'],
 ['despensa', 'desayunos', 'té'],
 ['despensa', 'desayunos', 'azúcaryendulzantes'],
 ['despensa', 'desayunos', 'cerealesyavena'],
 ['despensa', 'desayunos', 'vertodo'],
 ['despensa', 'lácteosyfrescos', 'lecheslíquidas'],
 ['despensa', 'lácteosyfrescos', 'lecheenpolvoysaborizantes'],
 ['despensa', 'lácteosyfrescos', 'vertodo'],
 ['despensa', 'mascotas', 'perros'],
 ['despensa', 'mascotas', 'gatos'],
 ['despensa', 'mascotas', 'vertodo'],
 ['despensa', 'aguasbebidasy

In [124]:
df_cat['category'] = filtered_lines

In [125]:
collection_cat_tottus = df_cat[df_cat['empresa']=='Tottus']['category'].values
collection_cat_lider = df_cat[df_cat['empresa']=='Lider']['category'].values
collection_cat_jumbo = df_cat[df_cat['empresa']=='Jumbo']['category'].values

In [126]:
def matching_cat(category, empresa, collection_cat_tottus, collection_cat_lider, collection_cat_jumbo):
    tottus = process.extractOne(category, collection_cat_tottus, scorer=fuzz.ratio)
    lider = process.extractOne(category, collection_cat_lider, scorer=fuzz.ratio)
    jumbo = process.extractOne(category, collection_cat_jumbo, scorer=fuzz.ratio)
    return([category, empresa, tottus[0], tottus[1], lider[0], lider[1], jumbo[0], jumbo[1]])

In [127]:
categories = df_cat[['category', 'empresa']].drop_duplicates().values

In [128]:
categories

array([['despensa abarrotes aceites', 'Tottus'],
       ['despensa abarrotes pastasysalsas', 'Tottus'],
       ['despensa abarrotes arrozlegumbresysemillas', 'Tottus'],
       ...,
       ['mascotas otrasmascotas alimentosotrasmascotas', 'Jumbo'],
       ['mascotas otrasmascotas accesoriosehigieneotrasmascotas',
        'Jumbo'],
       ['mascotas otrasmascotas vertodos', 'Jumbo']], dtype=object)

In [101]:
results = Parallel(n_jobs=-1, prefer="threads")(delayed(matching_cat)(category, empresa, collection_cat_tottus, collection_cat_lider, collection_cat_jumbo) for category, empresa in tqdm.tqdm(categories))

df_cat_res = pd.DataFrame(results, columns=['category', 'empresa', 'category_tottus', 'score_tottus', 'category_lider', 'score_lider', 'category_jumbo', 'score_jumbo'])

100%|██████████| 1482/1482 [04:36<00:00,  5.37it/s]


In [129]:
df_cat_res_0 = pd.merge(df_cat_res, df_cat[['category', 'empresa', 'first_level_category', 'second_level_category', 'third_level_category']].drop_duplicates(), how='left', on=['category', 'empresa'])
df_cat_res_0 = df_cat_res_0[['category', 'first_level_category', 'second_level_category', 'third_level_category' , 'empresa', 'category_tottus', 'score_tottus', 'category_lider', 'score_lider', 'category_jumbo', 'score_jumbo']]

In [130]:
df_cat_res_0

,category,first_level_category,second_level_category,third_level_category,empresa,category_tottus,score_tottus,category_lider,score_lider,category_jumbo,score_jumbo
0,despensa abarrotes aceites,Despensa,Abarrotes,Aceites,Tottus,despensa abarrotes aceites,100,despensa aceitesyaderezos aceites,71,despensa aceitessalycondimentos aceites,65
1,despensa abarrotes pastasysalsas,Despensa,Abarrotes,Pastas y Salsas,Tottus,despensa abarrotes pastasysalsas,100,despensa pastasysalsas vertodo,71,despensa fideospastasysalsas vertodos,70
2,despensa abarrotes arrozlegumbresysemillas,Despensa,Abarrotes,"Arroz, Legumbres y Semillas",Tottus,despensa abarrotes arrozlegumbresysemillas,100,despensa arrozylegumbres semillasycereales,74,despensa arrozlegumbresysemillas arroz,80
3,despensa abarrotes conservas,Despensa,Abarrotes,Conservas,Tottus,despensa abarrotes conservas,100,despensa conservas vertodo,67,despensa repostería coberturas,69
4,despensa abarrotes aderezosycondimentos,Despensa,Abarrotes,Aderezos y Condimentos,Tottus,despensa abarrotes aderezosycondimentos,100,despensa aceitesyaderezos condimentos,84,despensa aceitessalycondimentos sal,70
...,...,...,...,...,...,...,...,...,...,...,...
1477,mascotas gatos snacksysalsasgatos,Mascotas,Gatos,Snacks y Salsas Gatos,Jumbo,despensa mascotas gatos,50,mascotas gato accesoriosgato,72,mascotas gatos snacksysalsasgatos,100
1478,mascotas gatos vertodos,Mascotas,Gatos,Ver todos,Jumbo,despensa mascotas vertodo,67,mascotas gato vertodo,95,mascotas gatos vertodos,100
1479,mascotas otrasmascotas alimentosotrasmascotas,Mascotas,Otras Mascotas,Alimentos Otras Mascotas,Jumbo,despensa mascotas gatos,38,mascotas otrasmascotas accesoriosotrasmascotas,88,mascotas otrasmascotas alimentosotrasmascotas,100
1480,mascotas otrasmascotas accesoriosehigieneotras...,Mascotas,Otras Mascotas,Accesorios e Higiene Otras Mascotas,Jumbo,electroytecnología computación accesoriosdecom...,41,mascotas otrasmascotas accesoriosotrasmascotas,92,mascotas otrasmascotas accesoriosehigieneotras...,100


In [132]:
df_cat_final = df_cat_res_0.copy().drop_duplicates()
df_cat_final.head(3)

,category,first_level_category,second_level_category,third_level_category,empresa,category_tottus,score_tottus,category_lider,score_lider,category_jumbo,score_jumbo
0,despensa abarrotes aceites,Despensa,Abarrotes,Aceites,Tottus,despensa abarrotes aceites,100,despensa aceitesyaderezos aceites,71,despensa aceitessalycondimentos aceites,65
1,despensa abarrotes pastasysalsas,Despensa,Abarrotes,Pastas y Salsas,Tottus,despensa abarrotes pastasysalsas,100,despensa pastasysalsas vertodo,71,despensa fideospastasysalsas vertodos,70
2,despensa abarrotes arrozlegumbresysemillas,Despensa,Abarrotes,"Arroz, Legumbres y Semillas",Tottus,despensa abarrotes arrozlegumbresysemillas,100,despensa arrozylegumbres semillasycereales,74,despensa arrozlegumbresysemillas arroz,80


In [133]:
df_cat_final.shape

(1482, 11)

In [134]:
df_cat_final[['category', 'empresa']].drop_duplicates().shape

(1482, 2)

In [135]:
actual_date = str(datetime.now())[0:10]

df_cat_final.to_excel(ruta + f'df_cat_matching.xlsx'.format(actual_date), index=0)

In [136]:
umbral = 70

In [143]:
df_cat_final[df_cat_final['empresa']=='Tottus'].shape, df_cat_final[df_cat_final['empresa']=='Lider'].shape, df_cat_final[df_cat_final['empresa']=='Jumbo'].shape

((289, 11), (904, 11), (289, 11))

In [145]:
# Tottus vs Lider
df_cat_final[(df_cat_final['empresa']=='Tottus') & (df_cat_final['score_lider']>=umbral)].shape[0]

63

In [146]:
# Tottus vs Lider
df_cat_final[(df_cat_final['empresa']=='Tottus') & (df_cat_final['score_lider']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Tottus')].shape[0]

0.2179930795847751

In [148]:
# Tottus vs Jumbo
df_cat_final[(df_cat_final['empresa']=='Tottus') & (df_cat_final['score_jumbo']>=umbral)].shape[0]

20

In [149]:
# Tottus vs Jumbo
df_cat_final[(df_cat_final['empresa']=='Tottus') & (df_cat_final['score_jumbo']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Tottus')].shape[0]

0.06920415224913495

In [150]:
# Lider vs Tottus
df_cat_final[(df_cat_final['empresa']=='Lider') & (df_cat_final['score_tottus']>=umbral)].shape[0]

50

In [151]:
# Lider vs Tottus
df_cat_final[(df_cat_final['empresa']=='Lider') & (df_cat_final['score_tottus']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Lider')].shape[0]

0.05530973451327434

In [152]:
# Lider vs Jumbo
df_cat_final[(df_cat_final['empresa']=='Lider') & (df_cat_final['score_jumbo']>=umbral)].shape[0]

179

In [153]:
# Lider vs Jumbo
df_cat_final[(df_cat_final['empresa']=='Lider') & (df_cat_final['score_jumbo']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Lider')].shape[0]

0.19800884955752213

In [154]:
# Jumbo vs Tottus
df_cat_final[(df_cat_final['empresa']=='Jumbo') & (df_cat_final['score_tottus']>=umbral)].shape[0]

20

In [155]:
# Jumbo vs Tottus
df_cat_final[(df_cat_final['empresa']=='Jumbo') & (df_cat_final['score_tottus']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Jumbo')].shape[0]

0.06920415224913495

In [157]:
# Jumbo vs Lider
df_cat_final[(df_cat_final['empresa']=='Jumbo') & (df_cat_final['score_lider']>=umbral)].shape[0]

141

In [158]:
# Jumbo vs Lider
df_cat_final[(df_cat_final['empresa']=='Jumbo') & (df_cat_final['score_lider']>=umbral)].shape[0]/df_cat_final[(df_cat_final['empresa']=='Jumbo')].shape[0]

0.48788927335640137

In [111]:
df_final = pd.read_excel('df_matching.xlsx')
df_final.head(3)

,name,marca,formato,empresa,name_tottus,score_tottus,marca_tottus,formato_tottus,precio_tottus,name_lider,score_lider,marca_lider,formato_lider,precio_lider,name_jumbo,score_jumbo,marca_jumbo,formato_jumbo,precio_jumbo
0,ACEITE VEGETAL TOTTUS 900 CC,NaN,900 ml,Tottus,aceite vegetal tottus 900 cc 900 ml,100,NaN,900 ml,2000,aceite vegetal 900 ml romana 900 ml,74,la romana,900 ml,2890,aceite vegetal 900 ml natura 900 ml,74,natura,900 ml,2849
1,Aceite Vegetal,belmont,1 l,Tottus,aceite vegetal belmont 1 l,100,belmont,1 l,3940,aceite vegetal botella 1 l belmont 1 l,81,belmont,1 l,2590,aceite vegetal 1 l belmont 1 l,93,belmont,1 l,3299
2,Aceite Oliva Extra Virgen - 500 ML,tottus,500 ml,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ml,5490,aceite oliva extra virgen botella 500 ml olave...,83,olave,500 ml,6550,aceite oliva extra virgen 500 ml trattoria 500 ml,91,trattoria,500 ml,5299


In [112]:
df_final.shape

(15139, 19)

In [113]:
df_final_1 = df_final.drop_duplicates()
df_final_1.shape

(15139, 19)

In [114]:
df_final_2 = pd.merge(df_final_1, df[(df['empresa']=='Tottus') & (df['third_category']!='Ver Todo')][['product', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_tottus',
                                                                                                                                                                                        'first_category': 'first_category_tottus',
                                                                                                                                                                                        'second_category': 'second_category_tottus',
                                                                                                                                                                                        'third_category': 'third_category_tottus'}).drop_duplicates(), how='left', on='name_tottus').drop_duplicates()

In [115]:
df_final_2.shape

(20185, 22)

In [116]:
df_final_3 = pd.merge(df_final_2, df[(df['empresa']=='Lider') & (df['third_category']!='ver todo') & (df['third_category']!='Campañas')][['product', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_lider',
                                                                                                                                                                                        'first_category': 'first_category_lider',
                                                                                                                                                                                        'second_category': 'second_category_lider',
                                                                                                                                                                                        'third_category': 'third_category_lider'}).drop_duplicates(), how='left', on='name_lider').drop_duplicates()

In [117]:
df_final_3.shape

(23621, 25)

In [118]:
df_final_4 = pd.merge(df_final_3, df[(df['empresa']=='Jumbo') & (df['third_category']!='Ver todos') & (df['third_category']!='ver todo')][['product', 'first_category', 'second_category', 'third_category']].rename(columns={'product': 'name_jumbo',
                                                                                                                                                                                                                            'first_category': 'first_category_jumbo',
                                                                                                                                                                                                                            'second_category': 'second_category_jumbo',
                                                                                                                                                                                                                            'third_category': 'third_category_jumbo'}).drop_duplicates(), how='left', on='name_jumbo').drop_duplicates()

In [119]:
df_final_4.shape

(25034, 28)

In [120]:
df_final_4.head(3)

,name,marca,formato,empresa,name_tottus,score_tottus,marca_tottus,formato_tottus,precio_tottus,name_lider,...,precio_jumbo,first_category_tottus,second_category_tottus,third_category_tottus,first_category_lider,second_category_lider,third_category_lider,first_category_jumbo,second_category_jumbo,third_category_jumbo
0,ACEITE VEGETAL TOTTUS 900 CC,NaN,900 ml,Tottus,aceite vegetal tottus 900 cc 900 ml,100,NaN,900 ml,2000,aceite vegetal 900 ml romana 900 ml,...,2849,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites
1,Aceite Vegetal,belmont,1 l,Tottus,aceite vegetal belmont 1 l,100,belmont,1 l,3940,aceite vegetal botella 1 l belmont 1 l,...,3299,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites
2,Aceite Vegetal,belmont,1 l,Tottus,aceite vegetal belmont 1 l,100,belmont,1 l,3940,aceite vegetal botella 1 l belmont 1 l,...,3299,Despensa,Abarrotes,Pastas y Salsas,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites


In [121]:
actual_date = str(datetime.now())[0:10]

df_final_4.to_excel(ruta + f'/df_matching_categorias.xlsx'.format(actual_date), index=0)

In [122]:
df_final_4

,name,marca,formato,empresa,name_tottus,score_tottus,marca_tottus,formato_tottus,precio_tottus,name_lider,...,precio_jumbo,first_category_tottus,second_category_tottus,third_category_tottus,first_category_lider,second_category_lider,third_category_lider,first_category_jumbo,second_category_jumbo,third_category_jumbo
0,ACEITE VEGETAL TOTTUS 900 CC,NaN,900 ml,Tottus,aceite vegetal tottus 900 cc 900 ml,100,NaN,900 ml,2000,aceite vegetal 900 ml romana 900 ml,...,2849,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites
1,Aceite Vegetal,belmont,1 l,Tottus,aceite vegetal belmont 1 l,100,belmont,1 l,3940,aceite vegetal botella 1 l belmont 1 l,...,3299,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites
2,Aceite Vegetal,belmont,1 l,Tottus,aceite vegetal belmont 1 l,100,belmont,1 l,3940,aceite vegetal botella 1 l belmont 1 l,...,3299,Despensa,Abarrotes,Pastas y Salsas,Despensa,Aceites y Aderezos,Aceites,Despensa,"Aceites, Sal y Condimentos",Aceites
3,Aceite Oliva Extra Virgen - 500 ML,tottus,500 ml,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ml,5490,aceite oliva extra virgen botella 500 ml olave...,...,5299,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites de Oliva,Despensa,"Aceites, Sal y Condimentos",Aceites de Oliva
4,Aceite de Oliva Extra Virgen - 500 ML,tottus,500 ml,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100,tottus,500 ml,5490,aceite oliva extra virgen botella 500 ml olave...,...,5299,Despensa,Abarrotes,Aceites,Despensa,Aceites y Aderezos,Aceites de Oliva,Despensa,"Aceites, Sal y Condimentos",Aceites de Oliva
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25029,Pollo entero orgánico americano congelado kg,red wheelbarrow,congelado kg,Jumbo,alimento gato delimix cat chow 8 kg catchow al...,43,cat-chow,alimento seco 8 kg,31080,lomo centro americano congelado 25 kg eeuu 25 kg,...,21980,Despensa,Mascotas,Gatos,Carnes y Pescados,Todas las Carnes,Cortes Clásicos,Carnicería,Pollo,Pollo Entero
25030,Pollo entero orgánico americano congelado kg,red wheelbarrow,congelado kg,Jumbo,alimento gato delimix cat chow 8 kg catchow al...,43,cat-chow,alimento seco 8 kg,31080,lomo centro americano congelado 25 kg eeuu 25 kg,...,21980,Despensa,Mascotas,Gatos,Carnes y Pescados,Todas las Carnes,Carnes Americanas,Carnicería,Pollo,Pollo Entero
25031,Pollo entero orgánico americano congelado kg,red wheelbarrow,congelado kg,Jumbo,alimento gato delimix cat chow 8 kg catchow al...,43,cat-chow,alimento seco 8 kg,31080,lomo centro americano congelado 25 kg eeuu 25 kg,...,21980,Despensa,Mascotas,Gatos,Carnes y Pescados,Todas las Carnes,Pollo y Cerdo,Carnicería,Pollo,Pollo Entero
25032,Pollo entero orgánico americano congelado kg,red wheelbarrow,congelado kg,Jumbo,alimento gato delimix cat chow 8 kg catchow al...,43,cat-chow,alimento seco 8 kg,31080,lomo centro americano congelado 25 kg eeuu 25 kg,...,21980,Despensa,Mascotas,Gatos,Carnes y Pescados,Vacuno,Carnes Americanas,Carnicería,Pollo,Pollo Entero


In [54]:
df_res_cat = df_final_4.groupby(['first_category_tottus', 'second_category_tottus', 'third_category_tottus',
                                'first_category_lider', 'second_category_lider', 'third_category_lider',
                                'first_category_jumbo', 'second_category_jumbo', 'third_category_jumbo'], as_index=False).size().sort_values('size', ascending=False)

In [56]:
actual_date = str(datetime.now())[0:10]

df_res_cat.to_excel(ruta + f'/df_categorias.xlsx'.format(actual_date), index=0)